In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
from langchain_openai import ChatOpenAI


In [4]:
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_fireworks import ChatFireworks

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an essay assistant tasked with writing excellent 5-paragraph essays."
            " Generate the best essay possible for the user's request."
            " If the user provides critique, respond with a revised version of your previous attempts.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
llm = ChatOpenAI(model="gpt-3.5-turbo")

generate = prompt | llm

In [5]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk.content, end="")
    essay += chunk.content

"The Little Prince" by Antoine de Saint-Exupéry is a timeless classic that continues to captivate readers of all ages, especially children. Despite being written in the 1940s, the themes and messages in the story remain highly relevant in modern childhood. This essay will explore why "The Little Prince" continues to resonate with young readers today.

First and foremost, the Little Prince himself is a character that children can easily relate to. His innocent curiosity, genuine emotions, and unique perspective on the world mirror the way children see and experience the world around them. Children are drawn to the Little Prince's sense of wonder and his ability to ask meaningful questions about life, love, and friendship. In a world where children are often encouraged to grow up quickly, the Little Prince serves as a reminder to hold on to one's sense of wonder and imagination.

Moreover, the themes of friendship and connection portrayed in "The Little Prince" are particularly relevant 

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a teacher grading an essay submission. Generate critique and recommendations for the user's submission."
            " Provide detailed recommendations, including requests for length, depth, style, etc.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk.content, end="")
    reflection += chunk.content

Overall, your essay on why "The Little Prince" is relevant in modern childhood is well-written and insightful. You have effectively highlighted key themes and messages from the book and connected them to the experiences of children today. Here are some suggestions to enhance your essay further:

1. **Depth and Analysis**: While you touch upon various themes such as friendship, connection, loneliness, and imagination, consider delving deeper into each theme. Provide specific examples from the book that illustrate these themes and explain how they resonate with children today. Analyze how these themes can help children navigate their own lives and challenges.

2. **Expand on the Impact**: You mention that the Little Prince's relationships can teach children about forming connections and valuing uniqueness. Expand on this by discussing how these lessons can influence children's behavior and interactions with others. Provide real-life examples or studies that support the importance of thes

In [8]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(content=essay), HumanMessage(content=reflection)]}
):
    print(chunk.content, end="")

"The Little Prince" by Antoine de Saint-Exupéry remains a beloved classic that continues to resonate with modern children for a variety of reasons. One of the key themes of the book is the importance of friendship and connection. Through the Little Prince's interactions with characters like the fox and the rose, children learn about the value of forming meaningful relationships and cherishing the uniqueness of others. For example, the Little Prince's bond with the fox, who teaches him the true meaning of friendship through the ritual of taming, highlights the significance of genuine connections in a world that often prioritizes superficial interactions.

In addition to friendship, "The Little Prince" addresses complex emotions such as loneliness and loss, which are particularly relevant in modern childhood. Children today face various challenges that can lead to feelings of isolation and confusion. By following the Little Prince's journey of self-discovery and encountering characters l

In [9]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    return await generate.ainvoke({"messages": state})


async def reflection_node(messages: Sequence[BaseMessage]) -> List[BaseMessage]:
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    # First message is the original user request. We hold it the same for all nodes
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    res = await reflect.ainvoke({"messages": translated})
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res.content)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    if len(state) > 6:
        # End after 3 iterations
        return END
    return "reflect"


builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

In [10]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Generate an essay on the topicality of The Little Prince and its message in modern life"
        )
    ],
):
    print(event)
    print("---")

{'generate': AIMessage(content='"The Little Prince" by Antoine de Saint-Exupéry is a timeless classic that continues to resonate with readers of all ages due to its profound messages and themes. In today\'s fast-paced and materialistic world, the lessons from this novella are more relevant than ever. The story follows a young prince who travels from planet to planet, learning valuable life lessons along the way. One of the central messages of the book is the importance of seeing beyond the surface and cultivating meaningful connections with others.\n\nIn modern life, where social media and technology often dominate our interactions, the message of authentic human connections is particularly poignant. The Little Prince values the time he spends with the fox because of the bond they share, emphasizing the significance of relationships built on understanding and empathy. In a world where superficial connections are common, the book serves as a reminder to prioritize genuine relationships 

In [12]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

KeyError: '__end__'